In [89]:
import pandas as pd
from scipy.spatial import distance
from math import sin, cos, sqrt, atan2, radians
import time
import shapely
import fiona

from shapely.geometry import shape, Point
import shapely.wkt

from pyproj import Proj, transform

In [106]:
permits = pd.read_csv('data/nbrespermits.csv')
nypd2014_15 = pd.read_csv('../capstone_large_data_sets/nypd2014_15.csv')

In [107]:
permits.head()

,borough,bin_no,house_no,street_name,job_no,zip_code,job_start_date,owners_business_type,non-profit,latitude,longitude
0,MANHATTAN,1056547,2686,BROADWAY,121207354,10025,2022-05-11,PARTNERSHIP,N,40.798817,-73.968740
1,MANHATTAN,1812187,140,HILLSIDE AVENUE,121189524,10040,2022-05-11,CORPORATION,N,40.860296,-73.926125
2,BRONX,2823631,368,EAST 152 STREET,220586168,10455,2022-05-11,INDIVIDUAL,N,40.818565,-73.918118
3,BROOKLYN,3429007,3410,FARRAGUT ROAD,321588215,11210,2022-05-18,INDIVIDUAL,N,40.636513,-73.943944
4,BROOKLYN,3121674,1457,FLATBUSH AVENUE,321827163,11210,2022-05-11,CORPORATION,N,40.634773,-73.949721


In [108]:
nypd2014_15.head()

,cmplnt_num,cmplnt_fr_dt,law_cat_cd,boro_nm,addr_pct_cd,x_coord_cd,y_coord_cd,latitude,longitude
0,604271687,2015-03-01,FELONY,BRONX,48.0,1011779.0,246746.0,40.843901,-73.900505
1,990784784,2014-08-31,FELONY,BRONX,40.0,1009013.0,236135.0,40.814785,-73.910541
2,202268858,2015-01-01,MISDEMEANOR,BRONX,43.0,1020316.0,239179.0,40.823101,-73.869690
3,314003026,2015-04-01,MISDEMEANOR,MANHATTAN,34.0,1003518.0,249492.0,40.851460,-73.930354
4,117537458,2014-11-10,FELONY,QUEENS,114.0,1007654.0,219564.0,40.769306,-73.915508


## Breaking Data Up By Borough
Because the code to find the number of complaints within .1 miles takes a while to run, I broke the data up by borough. Ultimately, I broke Brooklyn's data up even further (see below).

In [111]:
permits_man = permits[permits['borough'] == 'MANHATTAN']
nypd2014_man = nypd2014_15[nypd2014_15['boro_nm'] == 'MANHATTAN']

permits_bx = permits[permits['borough'] == 'BRONX']
nypd2014_bx = nypd2014_15[nypd2014_15['boro_nm'] == 'BRONX']

permits_si = permits[permits['borough'] == 'STATEN ISLAND']
nypd2014_si = nypd2014_15[nypd2014_15['boro_nm'] == 'STATEN ISLAND']

permits_qn = permits[permits['borough'] == 'QUEENS']
nypd2014_qn = nypd2014_15[nypd2014_15['boro_nm'] == 'QUEENS']

permits_bk = permits[permits['borough'] == 'BROOKLYN']
nypd2014_bk = nypd2014_15[nypd2014_15['boro_nm'] == 'BROOKLYN']

In [112]:
permits_man.to_csv('boro_data/permits_man.csv', index = False)
nypd2014_man.to_csv('../big_data/nypd2014_man.csv', index = False)

permits_bx.to_csv('boro_data/permits_bx.csv', index = False)
nypd2014_bx.to_csv('../big_data/nypd2014_bx.csv', index = False)

permits_si.to_csv('boro_data/permits_si.csv', index = False)
nypd2014_si.to_csv('../big_data/nypd2014_si.csv', index = False)

permits_qn.to_csv('boro_data/permits_qn.csv', index = False)
nypd2014_qn.to_csv('../big_data/nypd2014_qn.csv', index = False)

permits_bk.to_csv('boro_data/permits_bk.csv', index = False)
nypd2014_bk.to_csv('../big_data/nypd2014_bk.csv', index = False)

### Splitting Brooklyn Up
I found I needed to split Brooklyn up to get a result. AWS kept breaking off what was running in my instance before the Brooklyn dataset could finish running.

In [114]:
permits_bk.shape

(2193, 11)

In [118]:
permits_bk1 = permits_bk[:731]
permits_bk2 = permits_bk[731:1462]
permits_bk3 = permits_bk[1462:]

In [119]:
print(permits_bk1.shape)
print(permits_bk2.shape)
print(permits_bk3.shape)

(731, 11)
(731, 11)
(731, 11)


In [120]:
permits_bk1.to_csv('boro_data/permits_bk1.csv', index = False)
permits_bk2.to_csv('boro_data/permits_bk2.csv', index = False)
permits_bk3.to_csv('boro_data/permits_bk3.csv', index = False)

# The Code for Complaints
I tried many iterations to speed it up and ultimately landed on this. This version of it is set up for a test on the first row of data.

I ultimately realized that seting a value for max_delt_lat and max_delt_long instead of using a variable, while making it harder to change that value quickly, would likely make the code a little faster.

[This stackoverflow](https://stackoverflow.com/questions/66470898/finding-pairs-of-latitude-and-longitude-within-a-certain-radius-in-python) was the first source I found for code on calculating distance between two points. It was largely sourced from this
[the stackoverflow](https://stackoverflow.com/questions/57294120/calculating-distance-between-latitude-and-longitude-in-python), which the first author cites. This second answer (combined with [this Earth fact sheet from Nasa](https://nssdc.gsfc.nasa.gov/planetary/factsheet/earthfact.html)) clarifies that R is the volumetric mean radius, or 6371 in km. This translates to 4182 miles.

I used 69 miles to the degree latitude, though it varies some.

At 48 degrees latitude, 1 degree longitude is about 40 miles. I used [this calculator](https://www.nhc.noaa.gov/gccalc.shtml) to find that.

In [124]:
def get_distance(point1, point2):
    R = 4182 #miles
    lat1 = radians(point1[0])  #insert value
    lon1 = radians(point1[1])
    lat2 = radians(point2[0])
    lon2 = radians(point2[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance


# note: the latitude and longitude I'm using are in decimals NOT minutes, so I do the same here
# these are set at a tenth of a mile to start, with 1 degree late = 69 miles and 1 degree longitude = 40 miles
max_delt_lat = 1/690
max_delt_long = 1/400


t0 = time.time()
for i in [0]: #<----THIS IS SET UP FOR A TEST
    lat = permits_man.loc[i, 'latitude']
    long = permits_man.loc[i, 'longitude']
    point1 = [lat, long]
    complaint_count = 0
    for n in nypd2014_man.index:
        n_lat = nypd2014_man.loc[n, 'latitude']
        n_long = nypd2014_man.loc[n, 'longitude']
        if (abs(lat - n_lat)<= max_delt_lat) or (abs(lat - n_long)<= max_delt_long):
            point2 = [n_lat, n_long]
            distance = get_distance(point1, point2)
            if distance <= .1:
                complaint_count += 1
    permits_man.loc[i, 'complaints2014_15'] = complaint_count

print('')
print("Time to run", time.time()-t0)


Time to run 1.582650899887085


/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_43538/82551024.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  permits_man.loc[i, 'complaints2014_15'] = complaint_count


### Separate `.py` Files For the Cloud
I took the above code and put it into `.py` files to run in AWS. I wound up needing more than one because AWS interrupted the file before it had finished producing all its outputs.

#### Some Notes on AWS
240 Manhattan entries took 871.3408896923065 seconds in the cloud.

I used Ubuntu and c3.8xlarge

This looks like it might be faster c5a.8xlarge

# Subway Entrances
I decided to look at numbers of entrances within tenth, half, one, and 2.5 mile radii of the permit. I wanted to find the closest entrance, along with half mile, but that would be very calculation intensive and take too long to run.

In [125]:
subways = pd.read_csv('data/subway_cleaned.csv')

In [126]:
subways.head()

,objectid,name,line,latitude,longitude
0,1734,Birchall Ave & Sagamore St at NW corner,2-5,40.849169,-73.868356
1,1735,Birchall Ave & Sagamore St at NE corner,2-5,40.849128,-73.868213
2,1736,Morris Park Ave & 180th St at NW corner,2-5,40.841223,-73.873499
3,1737,Morris Park Ave & 180th St at NW corner,2-5,40.841453,-73.872892
4,1738,Boston Rd & 178th St at SW corner,2-5,40.840815,-73.879623


In [130]:
permits.head()

,borough,bin_no,house_no,street_name,job_no,zip_code,job_start_date,owners_business_type,non-profit,latitude,longitude,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi
0,MANHATTAN,1056547,2686,BROADWAY,121207354,10025,2022-05-11,PARTNERSHIP,N,40.798817,-73.968740,5.0,15.0,42.0,195.0
1,MANHATTAN,1812187,140,HILLSIDE AVENUE,121189524,10040,2022-05-11,CORPORATION,N,40.860296,-73.926125,2.0,15.0,34.0,153.0
2,BRONX,2823631,368,EAST 152 STREET,220586168,10455,2022-05-11,INDIVIDUAL,N,40.818565,-73.918118,0.0,9.0,60.0,215.0
3,BROOKLYN,3429007,3410,FARRAGUT ROAD,321588215,11210,2022-05-18,INDIVIDUAL,N,40.636513,-73.943944,0.0,11.0,13.0,95.0
4,BROOKLYN,3121674,1457,FLATBUSH AVENUE,321827163,11210,2022-05-11,CORPORATION,N,40.634773,-73.949721,0.0,11.0,20.0,87.0


In [133]:
# note: the lati../backup_data/and longitude I'm using are in decimals NOT minutes, so I do the same here
# these are set at a tenth of a mile to start, with 1 degree late = 69 miles and 1 degree longitude = 40 miles
max_delt_lat = 1/690 #CHANGE HERE
max_delt_long = 1/400 #CHANGE HERE

t0 = time.time()
print("Tenth mile started ", t0)
permits.loc[i, 'subway_count_tenth_mi'] = 0 #CHANGE HERE
for i in permits.index:
    lat = permits.loc[i, 'latitude']
    long = permits.loc[i, 'longitude']
    point1 = [lat, long]
    entrance_count = 0
    for n in subways.index:
        n_lat = subways.loc[n, 'latitude']
        n_long = subways.loc[n, 'longitude']
        if (abs(lat - n_lat)<= max_delt_lat) or (abs(lat - n_long)<= max_delt_long):
            point2 = [n_lat, n_long]
            distance = get_distance(point1, point2)
            if distance <= .1: #CHANGE HERE
                entrance_count += 1
    permits.loc[i, 'subway_count_tenth_mi'] = entrance_count #CHANGE HERE

print("Time to run tenth mile:", time.time()-t0) #CHANGE HERE
print('')


max_delt_lat = 5/690 #CHANGE HERE
max_delt_long = 5/400 #CHANGE HERE

t0 = time.time()
print("Half mile started ", t0)
permits.loc[i, 'subway_count_half_mi'] = 0 #CHANGE HERE
for i in permits.index:
    lat = permits.loc[i, 'latitude']
    long = permits.loc[i, 'longitude']
    point1 = [lat, long]
    entrance_count = 0
    for n in subways.index:
        n_lat = subways.loc[n, 'latitude']
        n_long = subways.loc[n, 'longitude']
        if (abs(lat - n_lat)<= max_delt_lat) or (abs(lat - n_long)<= max_delt_long):
            point2 = [n_lat, n_long]
            distance = get_distance(point1, point2)
            if distance <= .5: #CHANGE HERE
                entrance_count += 1
    permits.loc[i, 'subway_count_half_mi'] = entrance_count #CHANGE HERE


print("Time to run half mile:", time.time()-t0) #CHANGE HERE
print('')


max_delt_lat = 10/690 #CHANGE HERE
max_delt_long = 10/400 #CHANGE HERE

t0 = time.time()
print("Mile started ", t0)
permits.loc[i, 'subway_count_one_mi'] = 0 #CHANGE HERE
for i in permits.index:
    lat = permits.loc[i, 'latitude']
    long = permits.loc[i, 'longitude']
    point1 = [lat, long]
    entrance_count = 0
    for n in subways.index:
        n_lat = subways.loc[n, 'latitude']
        n_long = subways.loc[n, 'longitude']
        if (abs(lat - n_lat)<= max_delt_lat) or (abs(lat - n_long)<= max_delt_long):
            point2 = [n_lat, n_long]
            distance = get_distance(point1, point2)
            if distance <= 1: #CHANGE HERE
                entrance_count += 1
    permits.loc[i, 'subway_count_one_mi'] = entrance_count #CHANGE HERE

print("Time to run one mile:", time.time()-t0) #CHANGE HERE
print('')

max_delt_lat = 25/690 #CHANGE HERE
max_delt_long = 25/400 #CHANGE HERE

t0 = time.time()
print("2.5 mile started ", t0)
permits.loc[i, 'subway_count_two_five_mi'] = 0 #CHANGE HERE
for i in permits.index:
    lat = permits.loc[i, 'latitude']
    long = permits.loc[i, 'longitude']
    point1 = [lat, long]
    entrance_count = 0
    for n in subways.index:
        n_lat = subways.loc[n, 'latitude']
        n_long = subways.loc[n, 'longitude']
        if (abs(lat - n_lat)<= max_delt_lat) or (abs(lat - n_long)<= max_delt_long):
            point2 = [n_lat, n_long]
            distance = get_distance(point1, point2)
            if distance <= 2.5: #CHANGE HERE
                entrance_count += 1
    permits.loc[i, 'subway_count_two_five_mi'] = entrance_count #CHANGE HERE

print("Time to run 2.5 mile:", time.time()-t0) #CHANGE HERE
print('')

Tenth mile started  1660092238.8299189
Time to run tenth mile: 151.76178216934204

Half mile started  1660092390.592167
Time to run half mile: 151.12384724617004

Mile started  1660092541.716223
Time to run one mile: 155.0053629875183

2.5 mile started  1660092696.721838
Time to run 2.5 mile: 156.59649896621704



In [134]:
permits['subway_count_tenth_mi'].value_counts()

0.0     6568
2.0      219
1.0      163
4.0       97
3.0       77
8.0       15
6.0       14
5.0       13
7.0        3
9.0        1
16.0       1
14.0       1
11.0       1
15.0       1
10.0       1
12.0       1
Name: subway_count_tenth_mi, dtype: int64

In [135]:
permits['subway_count_half_mi'].value_counts()

0.0     3150
2.0      407
4.0      394
8.0      306
6.0      306
        ... 
69.0       1
88.0       1
92.0       1
41.0       1
70.0       1
Name: subway_count_half_mi, Length: 70, dtype: int64

In [136]:
permits['subway_count_one_mi'].value_counts()

0.0      1862
3.0       291
5.0       251
4.0       210
8.0       205
         ... 
137.0       1
83.0        1
145.0       1
202.0       1
151.0       1
Name: subway_count_one_mi, Length: 161, dtype: int64

In [137]:
permits['subway_count_two_five_mi'].value_counts()

0.0      730
16.0     355
17.0     296
14.0     232
12.0     222
        ... 
372.0      1
398.0      1
481.0      1
525.0      1
246.0      1
Name: subway_count_two_five_mi, Length: 458, dtype: int64

In [138]:
permits.to_csv('../backup_data/permits_subway_backup.csv', index = False)

In [6]:
# permits = pd.read_csv('../backup_data/permits_subway_backup.csv')

# Historic Districts
Determing if a permit is located in a historic district.

In [139]:
permits.loc[0, 'point'] = 0

In [140]:
for i in permits.index:
    permits.loc[i, 'point'] = shapely.geometry.Point(permits.loc[i, 'longitude'], permits.loc[i, 'latitude'])

/Users/lorendunn/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:930: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return self.coerce_to_target_dtype(value).setitem(indexer, value)
/Users/lorendunn/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/managers.py:304: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  applied = getattr(b, f)(**kwargs)


In [141]:
permits.head()

,borough,bin_no,house_no,street_name,job_no,zip_code,job_start_date,owners_business_type,non-profit,latitude,longitude,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,hist_dist_name,in_hist_dist,point
0,MANHATTAN,1056547,2686,BROADWAY,121207354,10025,2022-05-11,PARTNERSHIP,N,40.798817,-73.968740,5.0,15.0,42.0,195.0,none,0.0,POINT (-73.96874 40.798817)
1,MANHATTAN,1812187,140,HILLSIDE AVENUE,121189524,10040,2022-05-11,CORPORATION,N,40.860296,-73.926125,2.0,15.0,34.0,153.0,none,0.0,POINT (-73.926125 40.860296)
2,BRONX,2823631,368,EAST 152 STREET,220586168,10455,2022-05-11,INDIVIDUAL,N,40.818565,-73.918118,0.0,9.0,60.0,215.0,none,0.0,POINT (-73.918118 40.818565)
3,BROOKLYN,3429007,3410,FARRAGUT ROAD,321588215,11210,2022-05-18,INDIVIDUAL,N,40.636513,-73.943944,0.0,11.0,13.0,95.0,none,0.0,POINT (-73.943944 40.636513)
4,BROOKLYN,3121674,1457,FLATBUSH AVENUE,321827163,11210,2022-05-11,CORPORATION,N,40.634773,-73.949721,0.0,11.0,20.0,87.0,none,0.0,POINT (-73.949721 40.634773)


In [142]:
for i in permits.index:
    permits.loc[i, 'hist_dist_name'] = 'none'

for i in permits.index:
    permits.loc[i, 'in_hist_dist'] = 0

## Used CSV With Latitude and Longitude Data for Historic Districts
I opted to use this instead of a shapefile because of the difficulty of converting latitude and longitude (WGS84) to NAD83. I wanted to be certain the geographic coordinates were all on the same scale.

This [stackoverflow answer](https://stackoverflow.com/a/18749373/5394724) showed me the basic approach to checking if a point is within a given area using shapely. I modified this because that answer was for one element and I have 154 historic districts, and because I used latitude and longitude from a .csv and this author used a shapefile. 

[This stackoverflow answer](https://stackoverflow.com/a/51892594/5394724) showed me how to make the shape using the long and latitude from the csv.

In [143]:
historic_dists = pd.read_csv('../historic_districts/LPC_HD_OpenData_2015Nov.csv')

In [144]:
historic_dists.columns

Index(['the_geom', 'BOROUGH', 'LP_NUMBER', 'CURRENT_', 'AREA_NAME',
       'EXTENSION', 'STATUS_OF_', 'LAST_ACTIO', 'BOUNDARY_N', 'PUBLIC_HEA',
       'OTHER_HEAR', 'DESDATE', 'CALDATE', 'Shape_Leng', 'Shape_Area'],
      dtype='object')

In [145]:
historic_dists.columns = historic_dists.columns.str.lower()

In [146]:
historic_dists.head()

,the_geom,borough,lp_number,current_,area_name,extension,status_of_,last_actio,boundary_n,public_hea,other_hear,desdate,caldate,shape_leng,shape_area
0,MULTIPOLYGON (((-73.78070476564865 40.79368791...,QN,LP-02040,Yes,Fort Totten Historic District,No,DESIGNATED,DESIGNATED,NaN,5/4/1999,NaN,6/29/1999,4/13/1999,11203.199722,4.067523e+06
1,MULTIPOLYGON (((-73.9545180987722 40.781621828...,MN,LP-01985,Yes,Hardenbergh/Rhinelander Historic District,No,DESIGNATED,DESIGNATED,NaN,3/10/1998,NaN,5/5/1998,NaN,410.860307,1.065242e+04
2,MULTIPOLYGON (((-74.00855983447059 40.71123470...,MN,LP-01901,Yes,African Burial Ground & The Commons Historic D...,No,DESIGNATED,DESIGNATED,NaN,9/1/1992,NaN,2/25/1993,NaN,4995.471462,1.069203e+06
3,MULTIPOLYGON (((-73.9595480661095 40.648261608...,BK,LP-00989,Yes,Albemarle-Kenmore Terraces Historic District,No,DESIGNATED,DESIGNATED,NaN,1/10/1978,NaN,7/11/1978,NaN,1240.770438,7.854686e+04
4,MULTIPOLYGON (((-73.96103396183 40.65868182663...,BK,LP-02567,Yes,Chester Court Historic District,No,DESIGNATED,DESIGNATED,AS IDENTIFIED BY RESEARCH DEPARTMENT (AND DETE...,11/25/2014,NaN,12/16/2014,10/28/2014,803.900855,3.840307e+04


In [147]:
t0 = time.time()

for i in range(len(historic_dists)):
    print(i)
    t1 = time.time()
    #shapefile_record = fiona_collection[i]
    shape = shapely.wkt.loads(historic_dists['the_geom'][i])
    minx, miny, maxx, maxy = shape.bounds
    bounding_box = shapely.geometry.box(minx, miny, maxx, maxy)
    for n in permits.index:
        if permits.loc[n, 'in_hist_dist'] != 1:
            point = permits.loc[n, 'point']
            if bounding_box.contains(point):
                if shape.contains(point):
                    permits.loc[n, 'in_hist_dist'] = 1
                    permits.loc[n, 'hist_dist_name'] = historic_dists.loc[i, 'area_name']
    print(f"Time from start to end of {i} loop: {time.time()-t0}")
    print(f"Time to run the {i} loop: {time.time()-t1}")
    print('')
print(f"Total time: {time.time()-t0}")

0
Time from start to end of 0 loop: 0.13018512725830078
Time to run the 0 loop: 0.12942099571228027

1
Time from start to end of 1 loop: 0.22683405876159668
Time to run the 1 loop: 0.09664106369018555

2
Time from start to end of 2 loop: 0.3256711959838867
Time to run the 2 loop: 0.09876203536987305

3
Time from start to end of 3 loop: 0.42232489585876465
Time to run the 3 loop: 0.09660696983337402

4
Time from start to end of 4 loop: 0.5207579135894775
Time to run the 4 loop: 0.09842300415039062

5
Time from start to end of 5 loop: 0.6146831512451172
Time to run the 5 loop: 0.09389209747314453

6
Time from start to end of 6 loop: 0.7095000743865967
Time to run the 6 loop: 0.09473490715026855

7
Time from start to end of 7 loop: 0.8058831691741943
Time to run the 7 loop: 0.0963740348815918

8
Time from start to end of 8 loop: 0.9001030921936035
Time to run the 8 loop: 0.0941622257232666

9
Time from start to end of 9 loop: 0.9984109401702881
Time to run the 9 loop: 0.0983119010925293



In [148]:
permits["in_hist_dist"].value_counts()

0.0    7134
1.0      42
Name: in_hist_dist, dtype: int64

In [149]:
permits.to_csv("../backup_data/perm_sub_hist_BACKUP.csv", index = False)